# Break Timer - "We need an Even Better AI Agent for that"

## Now with _Context Engineering_ (tm)

In [ ]:
from dotenv import load_dotenv
import time
from datetime import datetime
from IPython.display import Markdown, display
from agents import Agent, Runner
from pydantic import BaseModel, Field
load_dotenv(override=True)


In [ ]:

BREAK_MINS = 2
BREAK_SECS = BREAK_MINS * 60
BREAK_LENGTH = f"{BREAK_MINS} minutes"
MODEL = "gpt-4.1"

style = "in the excitable style of Steve Ballmer giving an employee pump talk"

now = datetime.now().strftime("%H:%M:%S")
print(f"The time is {now}")


In [ ]:
start_time = datetime.now()
start = start_time.strftime("%H:%M:%S")


agent_messages = []

instructions = f"""
You are an AI Agent responsible for timing a class break from a class about AI Agents.
The class is breaking for {BREAK_LENGTH} starting at {start}.
You are given the current time and you need to determine if the break is over.
If the break is over because {BREAK_LENGTH} has passed since {start}, you must respond only "Break is over!".
If the break is not over, you must respond with a hype-message {style} to keep the class engaged,
and to build up tension and excitement for when the class resumes! Be sure to mention how much time is left.
You will be given a list of your previous messages so you can keep the theme and add to the build-up.
"""

class Announcement(BaseModel):
    hype_message: str = Field(description=f"A hype-message in markdown to keep the class engaged, and to build up tension and excitement in the style {style} for when the class resumes!")
    is_break_over: bool = Field(description="Whether the break is over or not")

agent = Agent(name="Break Timer", instructions=instructions, model="gpt-4.1-mini", output_type=Announcement)


In [ ]:
break_over = False
while not break_over:
    now_time = datetime.now()
    now = now_time.strftime("%H:%M:%S")
    total_seconds_remaining = BREAK_SECS - (now_time - start_time).total_seconds()
    mins_remaining = total_seconds_remaining // 60
    seconds_remaining = total_seconds_remaining % 60
    input = f"The break began at {start}. The current time is {now}. The break should be a total of {BREAK_LENGTH} minutes. "
    if total_seconds_remaining > 0:
        input += f"That means there are {mins_remaining} minutes and {seconds_remaining} seconds remaining of the break.\n\n"
    else:
        input += "That means the break is over!\n\n"

    input += "\n\nRespond with your latest hype message in markdown and with whether the break is over or not."

    response = await Runner.run(agent,input)

    announcement = response.final_output_as(Announcement)

    display(Markdown(announcement.hype_message))

    if announcement.is_break_over:
        break_over = True
        print("Break is over!")
    else:
        agent_messages.append(announcement.hype_message)
        time.sleep(20)
